In [23]:
import torch
import torch.nn as nn
import torch.nn.parallel
from scripts import pix2pix


def set_requires_grad(params, flag):
    for p in params:
        p.requires_grad = flag

class RepeatChannel(nn.Module):
    def __init__(self, repeat):
        super(RepeatChannel, self).__init__()
        self.repeat = repeat

    def forward(self, img):
        return img.repeat(1, self.repeat, 1, 1)


class Downsample(nn.Module):
    def __init__(self, n_iter):
        super(Downsample, self).__init__()
        self.n_iter = n_iter

    def forward(self, img):
        for _ in range(self.n_iter):
            img = nn.functional.interpolate(img, scale_factor=0.5, mode='bicubic')
        return img


class Upsample(nn.Module):
    def __init__(self, n_iter):
        super(Upsample, self).__init__()
        self.n_iter = n_iter

    def forward(self, img):
        for _ in range(self.n_iter):
            img = nn.functional.interpolate(img, scale_factor=2.0, mode='bicubic')
        return img

class OutputTransform(nn.Module):
    def __init__(self, path, process='', diffaug_policy=''):
        super(OutputTransform, self).__init__()
        self.photosketch_path = path
        self.augment = None

        transforms = []
        process = process.split(',')
        for p in process:
            if p.startswith('up'):
                n_iter = int(p.replace('up', ''))
                transforms.append(Upsample(n_iter))
            elif p.startswith('down'):
                n_iter = int(p.replace('down', ''))
                transforms.append(Downsample(n_iter))
            elif p == 'to3ch':
                transforms.append(RepeatChannel(3))
            elif p == 'toSketch':
                sketch = self.setup_sketch(self.photosketch_path)
                transforms.append(sketch)
            else:
                ValueError("Transforms contains unrecognizable key: %s" % p)
        self.transforms = nn.Sequential(*transforms)

    def setup_sketch(self, photosketch_path):
        sketch = pix2pix.ResnetGenerator(3, 1, n_blocks=9, use_dropout=False)

        state_dict = torch.load(photosketch_path, map_location='cpu')
        if hasattr(state_dict, '_metadata'):
            del state_dict._metadata

        sketch.load_state_dict(state_dict)
        sketch.train()
        set_requires_grad(sketch.parameters(), False)
        return sketch

    def forward(self, img, apply_aug=True):
        img = self.transforms(img)
        return img

In [50]:
path = '/mnt/Data1/vmisra/Frida/scripts/pretrained/photosketch.pth'
t_real = 'toSketch'
tf_real = OutputTransform(path, process=t_real)

In [73]:
from PIL import Image

img_path = '/mnt/Data1/vmisra/Frida/scripts/frida.jpg'
img = Image.open(img_path)

In [78]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.PILToTensor()
])
width, height = img.size
img_tensor = transform(img).float()
img_tensor = img_tensor.reshape(1, 3, height, width)
# img_tensor = torch.from_numpy(img)

sketch = tf_real(img_tensor)

In [79]:
new_transform = transforms.Compose([
    transforms.ToPILImage()
])

sketch_test = new_transform(sketch[0])

In [2]:
cd src

/mnt/Data1/vmisra/Frida/src


In [1]:
%load_ext tensorboard

In [5]:
!python -m pip install -r requirements.txt             
!python -m pip install -r requirements_python3.txt     

  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 41.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 59.8 MB/s eta 0:00:0000:0100:01
  Using cached tensorboardX-2.1-py2.py3-none-any.whl (308 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 49.5 MB/s eta 0:00:0000:0100:01
  Using cached lpips-0.1.4-py3-none-any.whl (53 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 46.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.5 MB/s eta 0:00:00
  Using cached scikit_image-0.19.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.0 MB)
  Using cached roslibpy-1.3.0-py2.py3-none-any.whl (37 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 77.4 MB/s eta 0:00:00:00:0100:01
  Using cached future-0.18.2.tar.gz (829 kB)


In [3]:
import requests
import PIL
from io import BytesIO
def save_image_from_internet(url, fn):
    response = requests.get(url)
    img = PIL.Image.open(BytesIO(response.content))
    img = img.convert('RGB')
    img.save(fn)

In [9]:
image = "https://cdn.homeandmoney.com/wp-content/uploads/2022/05/31113751/Pittsburgh_FeaturedImg-1.jpg" #@param {type:"string"}

!wget $image
image = image.split('/')[-1]

%tensorboard --logdir src/painting 
!python plan.py --simulate --max_height 256 --use_cache --cache_dir cache/ --n_stroke_models 1 \
        --objective audio \
        --objective_data $image  \
        --objective_weight 1.0 \
        --lr_multiplier 0.4 \
        --n_init 0 \
        --num_strokes 800 --optim_iter 400 \
        --n_colors 30 \

--2022-10-31 00:33:21--  https://cdn.homeandmoney.com/wp-content/uploads/2022/05/31113751/Pittsburgh_FeaturedImg-1.jpg
Resolving cdn.homeandmoney.com (cdn.homeandmoney.com)... 108.156.172.51, 108.156.172.109, 108.156.172.44, ...
Connecting to cdn.homeandmoney.com (cdn.homeandmoney.com)|108.156.172.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 782594 (764K) [image/jpeg]
Saving to: ‘Pittsburgh_FeaturedImg-1.jpg.1’

Pittsburgh_Featured 100%[===================>] 764.25K  --.-KB/s    in 0.03s   

2022-10-31 00:33:21 (21.5 MB/s) - ‘Pittsburgh_FeaturedImg-1.jpg.1’ saved [782594/782594]



Reusing TensorBoard on port 6008 (pid 9743), started 0:14:42 ago. (Use '!kill 9743' to kill it.)